# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


A. Dumont  ->  A. Dumont  |  ['A. Dumont']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
Arxiv has 62 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.16288
extracting tarball to tmp_2601.16288... done.
Retrieving document from  https://arxiv.org/e-print/2601.16357


extracting tarball to tmp_2601.16357...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:488: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']


Issues with the citations
repeated bibliography entry: bhardwaj2020
Retrieving document from  https://arxiv.org/e-print/2601.16761
extracting tarball to tmp_2601.16761... done.
  1: tmp_2601.16761/main-alternate.tex, 486 lines
  2: tmp_2601.16761/main.tex, 411 lines
Retrieving document from  https://arxiv.org/e-print/2601.16977


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2601.16761/main-alternate.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'journals' from 'tmp_2601.16761/journals.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2601.16977... done.


Found 182 bibliographic references in tmp_2601.16977/reveallagn1.bbl.
Issues with the citations
syntax error in line 893: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.16523-b31b1b.svg)](https://arxiv.org/abs/2601.16523) | **On the Use of Field RR Lyrae as Galactic Probes -- VIII. Early Formation of the Galactic Spheroid**  |
|| G. Bono, et al. -- incl., <mark>M. Bergemann</mark> |
|*Appeared on*| *2026-01-26*|
|*Comments*| *Accepted for publication in ApJ. 62 pages, 29 figures, 10 tables*|
|**Abstract**|            We introduce a new photometric catalog of RR Lyrae variables (RRLs, $\sim$300,000) mainly based on data available in public datasets. We also present the largest and most homogeneous spectroscopic dataset of RRLs and Blue Horizontal Branch [BHB] stars ever collected. This includes radial velocity measurements ($\sim$16,000) and iron abundances ($\Delta$S method for 8,140 RRLs, plus 547 from literature). Elemental abundances based on high-resolution spectra are provided for 487 RRLs and 64 BHB stars. We identified candidate RRLs associated to the main Galactic components and their iron distribution function (IDF) becomes more metal-rich when moving from the Halo ([Fe/H]=-1.56) to the Thick (TCD; [Fe/H]=-1.47) and Thin (TND; [Fe/H]=-0.73) disk. Furthermore, Halo RRLs and RRLs in retrograde orbits are $\alpha$-enhanced ([$\alpha$/Fe]=0.27, $\sigma$=0.18), while TCD RRLs are either $\alpha$-enhanced ([Fe/H]$\le$-1.0) or $\alpha$-poor ([Fe/H]$>$-1.0), and TND RRLs are mainly $\alpha$-poor ([$\alpha$/Fe]=-0.01, $\sigma$=0.20). We also identified RRLs associated to the main stellar streams (Gaia-Sausage-Enceladus [GSE]; Sequoia, Helmi, Sagittarius) and we found that their IDFs are quite similar to Halo RRLs. However, GSE RRLs lack the metal-poor/metal-rich tails and their $\alpha$-element distribution is quite compact. The iron radial gradient in Galactocentric distance for TND, TCD and Halo RRLs is negative and it decreases from -0.026, to -0.010, and to -0.002 dex/kpc. The iron radial gradient based on dry Halo (Halo without substructures) RRLs is, within the errors, equal to the global Halo. We also found a strong similarity between iron and [$\alpha$/Fe] radial gradients of Milky Way RRLs and M31 globular clusters throughout the full range of galactocentric distances covered by the two samples.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.16977-b31b1b.svg)](https://arxiv.org/abs/2601.16977) | **ReveaLLAGN 1: JWST Emission-Line Spectra Reveal Low-Luminosity AGN with UV-Deficient SEDs and Warm Molecular Gas**  |
|| K. Goold, et al. -- incl., <mark>A. Dumont</mark>, <mark>N. Neumayer</mark> |
|*Appeared on*| *2026-01-26*|
|*Comments*| *Submitted to The Astrophysical Journal; under review after addressing referee comments. 25 pages, 10 figures*|
|**Abstract**|            We present near- and mid-infrared spectra of eight Low-Luminosity Active Galactic Nuclei (LLAGN), spanning nearly four orders of magnitude in black hole mass and Eddington ratio, obtained with JWST/NIRSpec and MIRI as part of the ReveaLLAGN program along with identical archival data of Cen A. The high spatial resolution of JWST cleanly separates AGN emission from host-galaxy contamination, enabling detections of high-ionization potential lines more than an order of magnitude fainter than previously measured. Emission-line diagnostics reveal a transition at log($L_{bol}/L_{Edd}$) ~ -3.5, where the spectral energy distribution becomes increasingly deficient in ultraviolet photons. We find that rotational H$_2$ excitation temperatures are elevated (~500 K higher) compared to both higher-luminosity AGN and star-forming galaxies, while the H$_2$(0-0)S(3)/PAH$_{11.3 \mu m}$ ratios are consistent with those observed in the AGN population. We discuss the possible roles of outflows, jets, and X-ray dominated regions in shaping the interstellar medium surrounding LLAGN. Silicate emission at ~10 $\mu$m, localized to the nuclear region, is detected in most ReveaLLAGN targets. This dataset offers the first comprehensive JWST-based characterization of infrared emission lines in the nuclear regions of LLAGN.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.16288-b31b1b.svg)](https://arxiv.org/abs/2601.16288) | **MeerKAT observations of Abell 1775 and Abell 1795: the discovery of a hadronic radio halo?**  |
|| R. J. v. Weeren, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-01-26*|
|*Comments*| *22 pages, 15 figures, accepted for publication in MNRAS*|
|**Abstract**|            Giant radio haloes are diffuse synchrotron sources typically found in merging galaxy clusters, while smaller mini-haloes occur in cool-core clusters. Both trace cosmic-ray electrons in the intracluster medium, though recent observations suggest their distinction is not always clear. We present new 903-1655 MHz MeerKAT observations of Abell 1775 and Abell 1795, both hosting cool cores and cold fronts. Combined with reprocessed 120-168 MHz LOFAR Two-metre Sky Survey data, we perform imaging and spectral analyses of their radio emission. In both clusters, we detect radio haloes with distinct inner and outer components. In Abell 1775, the halo appears diffuse at 1.3 GHz, while LOFAR images reveal steep-spectrum filaments. In Abell 1795, the inner component corresponds to a previously reported mini-halo candidate, but the full structure extends to $\sim$1 Mpc with a spectral index of $\alpha=-1.08\pm0.06$. The presence of such a large, flat-spectrum halo in a dynamically relaxed cluster makes Abell 1795 an outlier relative to typical merging systems. This suggests that some relaxed clusters may still retain sufficient turbulence to sustain particle re-acceleration, or that hadronic interactions producing secondary electrons play a significant role. Together with other recent discoveries in cool-core systems, our results indicate that some large radio haloes may have been overlooked in past studies due to limited dynamic range near bright central AGN. Finally, we detect steep-spectrum emission south of Abell 1795's central AGN, tracing a 45 kpc X-ray and optical filament that terminates in an X-ray cavity, likely linked to a past AGN outburst.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.16761-b31b1b.svg)](https://arxiv.org/abs/2601.16761) | **Illuminating the Physics of Cosmic Origin and Evolution: A UK Space Frontiers 2035 White Paper**  |
|| F. Beutler, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-01-26*|
|*Comments*| *White paper submitted to the UK Space Agency's initiative "UK Space Frontiers 2035" this https URL*|
|**Abstract**|            Understanding the Universe's origins and evolution remains one of the most fundamental challenges in modern cosmology. This white paper explores three key science priorities in this field: unravelling the physics of cosmic inflation, investigating the accelerating expansion of the Universe, and precisely measuring the sum of the neutrino masses. Achieving these goals requires a dedicated survey to map the large-scale structure at high redshift in unprecedented detail. We describe how this can be achieved through a mission concept called SIRMOS, providing a high-throughput, highly multiplexed spectroscopic capability to obtain accurate redshifts for over 100 million galaxies over a wide sky area. Such a survey would leverage the deepest existing wide-area photometric catalogues for targeting, with spectra offering continuous 1.25-2.5~$\mu$m wavelength coverage at moderate resolution, allowing precise redshift measurements in the $1<z<4$ range with minimal bias. We outline the scientific opportunities this presents. Recent years have seen significant advances in instrumentation, including digital micromirror devices, complex telescope mirrors, large detector arrays, and data processing pipelines. While these technologies have been demonstrated in terrestrial applications, such a survey is a unique opportunity to apply these proven capabilities in space to address fundamental questions in cosmology. Participation in such a mission will simultaneously deliver a compelling science case, help align UK Space Agency and STFC strategies, demonstrate the UK's growing capability in end-to-end space missions, and strengthen the national space economy through high-value industrial participation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.16357-b31b1b.svg)](https://arxiv.org/abs/2601.16357) | **TOI-6692b: An eccentric 130 day period giant planet with a single transit from TESS**  |
|| A. Bieryla, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2026-01-26*|
|*Comments*| *13 pages, 9 figures, accepted 22 Jan 2026*|
|**Abstract**|            We report the discovery and characterization of TOI-6692 b, an eccentric (e~0.54) Jupiter on a 130-day orbit. TOI-6692 b was first detected as a community TESS Object of Interest (cTOI) by the Visual Survey Group and the Planet Hunters group as a single transit candidate via TESS observation. The period was subsequently confirmed via radial velocity monitoring from the Planet Finder Spectrograph on the 6.5m Magellan telescope. Additional radial velocities were acquired with the CHIRON, FEROS, and CORALIE spectrographs. LCOGT ground-based photometric follow-up was conducted over 2 weeks to detect another transit and refine the period. Although we did not detect an ingress or egress of the 11.04 hr transit, we did detect a possible in-transit signal in the multi-night data and provide an updated ephemeris for future monitoring. TOI-6692 b is one of few planets with orbital periods longer than 100 days that have a secure mass, radius, and eccentricity detection. As with most giant planets at these orbital periods, the eccentricity of TOI-6692 b is lower than that expected of planets undergoing high-eccentricity tidal migration, but is more consistent with the expectations of planet-planet scattering outcomes. A long-term radial velocity trend was detected and further monitoring is warranted to determine the outer companion period. TOI-6692 b is also one of few TESS single transit targets that have its period eventually confirmed via follow-up photometric campaigns timed to capture transits despite the relatively large ephemeris uncertainties. Such efforts highlight the capabilities of night-to-night stability on ground-based photometric facilities today.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.16523/./gal_dynamic.png', 'tmp_2601.16523/./stream_dynamic.png', 'tmp_2601.16523/./iron_radial_gradient.png']
copying  tmp_2601.16523/./gal_dynamic.png to _build/html/
copying  tmp_2601.16523/./stream_dynamic.png to _build/html/
copying  tmp_2601.16523/./iron_radial_gradient.png to _build/html/
exported in  _build/html/2601.16523.md
    + _build/html/tmp_2601.16523/./gal_dynamic.png
    + _build/html/tmp_2601.16523/./stream_dynamic.png
    + _build/html/tmp_2601.16523/./iron_radial_gradient.png
found figures ['tmp_2601.16977/./shock_diagrams.png', 'tmp_2601.16977/./L_NEVvsL_OIV.png', 'tmp_2601.16977/./Median_Flux_Ratio.png']
copying  tmp_2601.16977/./shock_diagrams.png to _build/html/
copying  tmp_2601.16977/./L_NEVvsL_OIV.png to _build/html/
copying  tmp_2601.16977/./Median_Flux_Ratio.png to _build/html/
exported in  _build/html/2601.16977.md
    + _build/html/tmp_2601.16977/./shock_diagrams.png
    + _build/html/tmp_2601.16977/./L_NEVvsL_OIV.png
    + _build/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\fmmm}[1]{\mbox{#1}}$
$\newcommand{\scnd}{\mbox{\fmmm{"}\hskip-0.3em .}}$
$\newcommand{\scnp}{\mbox{\fmmm{"}}}$
$\newcommand{\mcnd}{\mbox{\fmmm{'}\hskip-0.3em .}}$
$\newcommand{\lsun}{\hbox{L_{\odot}}}$
$\newcommand{\msun}{\hbox{M_{\odot}}}$
$\newcommand{\rsun}{\hbox{R_{\odot}}}$
$\newcommand{\teff}{\hbox{T_{\mathrm{eff}}}}$
$\newcommand{\logg}{\hbox{\log g}}$
$\newcommand{\feh}{\hbox{[Fe/H]}}$
$\newcommand{\mh}{\hbox{[M/H]}}$
$\newcommand{\xh}{\hbox{[X/H]}}$
$\newcommand{\kms}{\hbox{km s^{-1}}}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\erf}{\mathop{\rm erf}\nolimits}$
$\newcommand{\sech}{ \mathop{\rm sech}\nolimits}$
$\newcommand{\csch}{ \mathop{\rm csch}\nolimits}$
$\newcommand{\arcsinh}{\mathop{\rm arcsinh}\nolimits}$
$\newcommand{\arccosh}{\mathop{\rm arccosh}\nolimits}$
$\newcommand{\arctanh}{\mathop{\rm arctanh}\nolimits}$
$\newcommand{\arccoth}{\mathop{\rm arccoth}\nolimits}$
$\newcommand{\arcsech}{\mathop{\rm arcsech}\nolimits}$
$\newcommand{\arccsch}{\mathop{\rm arccsch}\nolimits}$
$\newcommand{\arccot}{\mathop{\rm arccot}\nolimits}$
$\newcommand{\arcsec}{\mathop{\rm arcsec}\nolimits}$
$\newcommand{\arccsc}{\mathop{\rm arccsc}\nolimits}$
$\newcommand{\ylm}{\mathop{\rm Y}_l^m\nolimits}$
$\newcommand{\ylmp}{\mathop{\rm Y}_{l'}^{m'}\nolimits}$
$\newcommand{\real}{\Re e}$
$\newcommand{\imag}{\Im m}$
$\newcommand{\ltsim}{   ^<_\sim  }$
$\newcommand{\deg}{^{\circ}}$
$\newcommand{\farcm}{\hbox{\mkern-4mu^\prime}}$
$\newcommand{\parcm}{ ^{\prime}\llap{.}}$
$\newcommand{\farcs}{\hbox{^{\prime\prime}}~}$
$\newcommand{\parcs}{ ^{\prime\prime}\llap{.}}$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\masyr}{{\rm mas/yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\g}{{\rm g}}$
$\newcommand{\om}{\Omega_0}$
$\newcommand\ca{{\bf ca.\/}}$
$\newcommand\r{r^{1/4} }$
$\newcommand\magnitude{^{\rm m}}$
$\newcommand{\kr}{{\cal K}_r}$
$\newcommand{\kz}{{\cal K}_z}$
$\newcommand{\kzz}{{\cal K}_z(z)}$
$\newcommand{\mss}{{\rm M}_\odot \rm pc^{-2}}$
$\newcommand{\msss}{{\rm M}_\odot \rm pc^{-3}}$
$\newcommand{\Aa}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{Å}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\ltsim}{   ^<_\sim  }$
$\newcommand{\gtsim}{   ^>_\sim  }$
$\newcommand{\hr}{ ^{\footnotesize h}}$
$\newcommand{\tm}{ ^{\footnotesize m}}$
$\newcommand{\ts}{ ^{\footnotesize s}}$
$\newcommand{\Ts}{ ^{\footnotesize s}\llap{.}}$
$\newcommand{\Deg}{ ^\circ\llap{.}}$
$\newcommand{\Min}{ ^{\prime}\llap{.}}$
$\newcommand{\Sec}{ ^{\prime\prime}\llap{.}}$
$\newcommand{\deg}{ ^\circ}$
$\newcommand{\min}{ ^{\prime}}$
$\newcommand{\sec}{ ^{\prime\prime}}$
$\newcommand{\gtsim}{   ^>_\sim  }$
$\newcommand{\hst}{{\bf HST\/}}$
$\newcommand{\bmv}{\hbox{\it B--V\/}}$
$\newcommand{\bmr}{\hbox{\it B--R\/}}$
$\newcommand{\bmi}{\hbox{\it B--I\/}}$
$\newcommand{\vmi}{\hbox{\it V--I\/}}$
$\newcommand{\umi}{\hbox{\it U--I\/}}$
$\newcommand{\jmh}{\hbox{\it J--H\/}}$
$\newcommand{\jmk}{\hbox{\it J--K\/}}$
$\newcommand{\hmk}{\hbox{\it H--K\/}}$
$\newcommand{\bmk}{\hbox{\it B--K\/}}$
$\newcommand{\vmr}{\hbox{\it V--R\/}}$
$\newcommand{\rmi}{\hbox{\it R--I\/}}$
$\newcommand{\bmj}{\hbox{\it B--J\/}}$
$\newcommand{\vmj}{\hbox{\it V--J\/}}$
$\newcommand{\rmj}{\hbox{\it R--J\/}}$
$\newcommand{\imj}{\hbox{\it I--J\/}}$
$\newcommand{\bmh}{\hbox{\it B--H\/}}$
$\newcommand{\vmh}{\hbox{\it V--H\/}}$
$\newcommand{\rmh}{\hbox{\it R--H\/}}$
$\newcommand{\imh}{\hbox{\it I--H\/}}$
$\newcommand{\vmk}{\hbox{\it V--K\/}}$
$\newcommand{\rmk}{\hbox{\it R--K\/}}$
$\newcommand{\imk}{\hbox{\it I--K\/}}$
$\newcommand\ngc{#1}$
$\newcommand{\vs}{\textit{vs}\/}$</div>



<div id="title">

# On the Use of Field RR Lyrae as Galactic Probes - VIII. Early Formation of the Galactic Spheroid$\footnote{Based on observations collected at the European Southern Observatory (ESO) with the ERIS spectrograph available at the Very Large Telescope (VLT), Paranal Observatory, Chile (ESO programs: 0111.D-0233(A), 0112.D-2235(A), 0113.D-2226(A), PI: G. Bono), and with WINERED spectrograph available as a visitor instrument at the ESO New Technology Telescope (NTT), La Silla, Chile (ESO program: 098.D-0878(A), PI: G. Bono).Based in part on observations made with the Southern African Large Telescope (SALT):Program IDs: 2017-2-SCI-041, 2018-1-SCI-018, 2018-2-SCI-025, 2019-1-SCI-013, 2021-2-SCI-028, 2022-2-DDT-001, PI: B. Chaboyer).Based in part on data obtained in the Observatorios de Canarias del Instituto de Astrofisica de Canarias (IAC) with: the STELLA robotic telescope, an AIP facility jointly operated by AIP and IAC at the Teide Observatory in Tenerife, Spain; the Italian Telescopio Nazionale Galileo (TNG) operated by the Fundaci{ó}n Galileo Galilei of the INAF, the Nordic Optical Telescope (NOT) owned in collaboration by the University of Turku and Aarhus University and operated jointly by Aarhus University, the University of Turku and the University of Oslo, representing Denmark, Finland and Norway, the University of Iceland and Stockholm University, and the Mercator telescope operated by the Flemish Community, all at the Observatorio del Roque de los Muchachos of the IAC, La Palma, Spain (Program IDs: 101-MULTIPLE-4/21B, 107-MULTIPLE-4/22A, 120-MULTIPLE-2/23B, 165-Stella12/20A, PI: M. Monelli; Program ID: 108-MULTIPLE-2/25B, PI: M. S{á}nchez-Benavente).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.16523-b31b1b.svg)](https://arxiv.org/abs/2601.16523)<mark>Appeared on: 2026-01-26</mark> -  _Accepted for publication in ApJ. 62 pages, 29 figures, 10 tables_

</div>
<div id="authors">

G. Bono, et al. -- incl., <mark>M. Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** We introduce a new photometric catalog of RR Lyrae variables(RRLs, $\sim$ 300,000) mainly based on data available in public datasets.We also present the largest and most homogeneous spectroscopic dataset of RRLsand Blue Horizontal Branch [ BHB ] stars ever collected.This includes radial velocity measurements ( $\sim$ 16,000)and iron abundances ( $\Delta$ S method for 8,140 RRLs, plus $547$ from literature).Elemental abundances based on high-resolution spectra are provided for 487 RRLs and 64 BHB stars.We identified candidate RRLs associated to the main Galactic components and their iron distributionfunction (IDF) becomes more metal-rich when moving from the Halo ( [ Fe/H ] =-1.56) to theThick (TCD; [ Fe/H ] =-1.47) and Thin (TND; [ Fe/H ] =-0.73) disk. Furthermore, Halo RRLsand RRLs in retrograde orbits are $\alpha$ -enhanced ( [ $\alpha$ /Fe ] =0.27, $\sigma$ =0.18),while TCD RRLs are either $\alpha$ -enhanced ( [ Fe/H ] $\le$ -1.0) or $\alpha$ -poor ( [ Fe/H ] $>$ -1.0),and TND RRLs are mainly $\alpha$ -poor ( [ $\alpha$ /Fe ] =-0.01, $\sigma$ =0.20).We also identified RRLs associated to the main stellar streams (Gaia-Sausage-Enceladus [ GSE ] ; Sequoia,Helmi, Sagittarius) and we found that their IDFs are quite similar to Halo RRLs. However,GSE RRLs lack the metal-poor/metal-rich tails and their $\alpha$ -element distribution is quite compact.The iron radial gradient in Galactocentric distance for TND, TCD and Halo RRLs is negative and itdecreases from -0.026, to -0.010, and to -0.002 dex/kpc. The iron radial gradient based on dry Halo(Halo without substructures) RRLs is, within the errors, equal to the global Halo.We also found a strong similarity between iron and [ $\alpha$ /Fe ] radial gradients of Milky Way RRLsand M31 globular clusters throughout the full range of galactocentric distances covered by the two samples.

</div>

<div id="div_fig1">

<img src="tmp_2601.16523/./gal_dynamic.png" alt="Fig10" width="100%"/>

**Figure 10. -** Kinematic planes for RRLs in the main Galactic components.
Panel a)--[Toomre diagram]: RRL transversal velocity as a function of the sum in
quadrature of radial and vertical velocity components ($\sqrt{V^2_R+V^2_Z}$).
TND and TCD RRLs are marked with green and orange dots, while Bulge RRLs
are marked with magenta dots. The shaded gray area shows the distribution of Halo
RRLs and RRLs in retrograde orbits. RRLs in the different Galactic components
were selected according to \citet{bensby2014}, to circularity (retrograde),
and to apocentric radius (Bulge). See text for more details.
Panel b): RRL transversal velocity as a function of the RV ($V_T$\vs$V_R$).
Panel c)--[Lindblad diagram]: RRL orbital energy as a function of the vertical
component of the angular momentum ($E$\vs $L_Z$).
Panel d):  Eccentricity $\vs$$L_Z$.
Panel e): the square root of the radial action ($\sqrt{J_R}$) $\vs$$J_\phi=L_Z$.
Panel f)--[Action diamond diagram]: $(J_Z-J_R)/J_{tot}$\vs$J_\phi/J_{tot}$,
where $J_{tot}=|J_\phi|+J_R+J_Z$).
Panel g): RRL circularity as a function of eccentricity.
Panel h): RRL circularity as a function of the maximum height above the Galactic
plane (logarithmic axis).
Panel i): RRL circularity as a function of the Galactocentric distance
(logarithmic axis).
 (*fig:gal_dynamic*)

</div>
<div id="div_fig2">

<img src="tmp_2601.16523/./stream_dynamic.png" alt="Fig11" width="100%"/>

**Figure 11. -** Kinematic planes for RRLs in the four selected
stellar streams. The green dots mark candidate GSE RRLs, orange
dots display candidate SEQ1 and SEQ2 RRLS according to \citet{feuillet2021}.
The turquoise dots mark Helmi RRLs, according to \citet{koppelman2019a} and \citet{Horta2023}.
The red dots show candidate SGR RRLs according to \citet{antoja2020}.
The gray shaded area shows the same RRLs belonging to the main Galactic
components plotted in Fig. \ref{fig:gal_dynamic} (*fig:stream_dynamic*)

</div>
<div id="div_fig3">

<img src="tmp_2601.16523/./iron_radial_gradient.png" alt="Fig17" width="100%"/>

**Figure 17. -** Panels a,b,c -- Iron abundance for TND RRLs as a function of Galactocentric
distance (panel a), distance projected onto the Galactic plane (panel b), and
maximum height above the Galactic plane (panel c).  The X-axes are in logarithmic
scale. The blue lines display the running average, while the orange lines the
standard deviation of the running average. The dashed-dotted lines show
the linear fit over the different sub-samples. The insets display the standard
deviation of the spectroscopic sample as a function of distance.
Panels d,e,f -- Same as the top, but for TCD RRLs.
Panels g,h,i -- Same as the top, but for Halo+Retro RRLs.
Panels j,k,l -- Same as the top, but for dry Halo+Retro RRLs. See text for more details.
 (*fig:iron_radial_gradient*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.16523"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\aastex}{AAS\TeX}$
$\newcommand{\latex}{La\TeX}$
$\newcommand{\nefive}{[\ion{Ne}{5}]~\ensuremath{\lambda14.32~\mu\mathrm{m}}}$
$\newcommand{\ofour}{[\ion{O}{4}]~\ensuremath{\lambda25.91~\mu\mathrm{m}}}$
$\newcommand{\nethree}{[\ion{Ne}{3}]~\ensuremath{\lambda15.56~\mu\mathrm{m}}}$
$\newcommand{\netwo}{[\ion{Ne}{2}]~\ensuremath{\lambda12.81~\mu\mathrm{m}}}$
$\newcommand{\fetwo}{[\ion{Fe}{2}]~\ensuremath{\lambda5.34~\mu\mathrm{m}}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\lbolledd}{\ensuremath{\log(L_{\mathrm{bol}}/L_{\mathrm{Edd}})}}$</div>



<div id="title">

# ReveaLLAGN 1: JWST Emission-Line Spectra Reveal Low-Luminosity AGN with UV-Deficient SEDs and Warm Molecular Gas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.16977-b31b1b.svg)](https://arxiv.org/abs/2601.16977)<mark>Appeared on: 2026-01-26</mark> -  _Submitted to The Astrophysical Journal; under review after addressing referee comments. 25 pages, 10 figures_

</div>
<div id="authors">

K. Goold, et al. -- incl., <mark>A. Dumont</mark>, <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** We present near- and mid-infrared spectra of eight Low-Luminosity Active Galactic Nuclei (LLAGN), spanning nearly four orders of magnitude in black hole mass and Eddington ratio, obtained with JWST/NIRSpec and MIRI as part of the ReveaLLAGN program along with identical archival data of Cen A. The high spatial resolution of JWST cleanly separates AGN emission from host-galaxy contamination, enabling detections of high–ionization potential lines more than an order of magnitude fainter than previously measured. Emission-line diagnostics reveal a transition at log( $L_{\rm bol}/L_{\rm Edd}$ ) $\sim -3.5$ , where the spectral energy distribution becomes increasingly deficient in ultraviolet photons. We find that rotational $H_2$ excitation temperatures are elevated ( $\sim$ 500 K higher) compared to both higher-luminosity AGN and star-forming galaxies, while the $H_2$ (0-0)S(3)/PAH $_{11.3 \mu \mathrm{m}}$ ratios are consistent with those observed in the AGN population. We discuss the possible roles of outflows, jets, and X-ray dominated regions in shaping the interstellar medium surrounding LLAGN. Silicate emission at $\sim$ 10 $\mu$ m, localized to the nuclear region, is detected in most ReveaLLAGN targets. This dataset offers the first comprehensive JWST-based characterization of infrared emission lines in the nuclear regions of LLAGN.

</div>

<div id="div_fig1">

<img src="tmp_2601.16977/./shock_diagrams.png" alt="Fig10" width="100%"/>

**Figure 10. -** The emission line ratios observed in ReveaLLAGN targets are broadly consistent with photoionization models using a power-law ionizing spectrum (red grid; \citealt{Fern2023}). For NGC 4395, which has a relatively low black hole mass and accretes above a log Eddington ratio of –2.7, the line ratios are better reproduced by accretion disk–dominated photoionization models (blue grid; \citealt{Ferland2020}). Shock-only models (black grid) reproduce neon line ratios that are degenerate with those from power-law photoionization models, but only within a narrow range of low pre-shock densities and low shock velocities. At higher pre-shock densities, the shock models become inconsistent with the observations, and they fail to reproduce several key line ratios beyond neon. Gray squares represent comparison data as described in \ref{sec:supplemental_data}, and ReveaLLAGN sources are colored the same as in Figure \ref{fig:nev_oiv}. (*fig:photo_vs_shock*)

</div>
<div id="div_fig2">

<img src="tmp_2601.16977/./L_NEVvsL_OIV.png" alt="Fig2" width="100%"/>

**Figure 2. -** [NeV]$_{14 \mu \rm{m}}$ luminosity versus [OIV]$_{26 \mu \rm{m}}$. ReveaLLAGN targets all show low-luminosity detections in both lines, with M94 being the faintest. For comparison, data from previous surveys \citet{Sturm2002, Goulding2009, Tommasin2010, Fernandez2016} are shown in gray. 1$\sigma$ errors are smaller than data points. [NeV]$_{14 \mu \rm{m}}$ and [OIV]$_{26 \mu \rm{m}}$ are strongly correlated across many orders of magnitude in solar luminosity, and the ReveaLLAGN sample extends this trend to the lowest luminosities. ReveaLLAGN sources are ordered in the legend according to Eddington ratio and colored using the same sequential color map as in Figure \ref{fig:nuc_large_ratio}. (*fig:nev_oiv*)

</div>
<div id="div_fig3">

<img src="tmp_2601.16977/./Median_Flux_Ratio.png" alt="Fig1" width="100%"/>

**Figure 1. -** Emission from the nuclear extracted spectra is consistent with that of a point-like source. The dashed line marks the ratio expected for an unresolved point source. For each galaxy, we show the median and standard deviation of the flux ratios computed across all detected ionic lines. Data points are ordered, left to right, by Eddington ratio and colored using a sequential color scheme carried through the rest of the paper. (*fig:nuc_large_ratio*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.16977"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

114  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
